In [0]:
import cv2
import numpy as np
from keras_squeezenet import SqueezeNet
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D
from keras.models import Sequential
import tensorflow as tf
import os

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [55]:
CLASS_MAP = dict()
reverselookup = dict()
count = 0
for j in os.listdir('drive/My Drive/Data/BGR'):
    if not j.startswith('.'): # If running this code locally, this is to 
                              # ensure you aren't reading in hidden folders
        CLASS_MAP[j] = count
        reverselookup[count] = j
        count = count + 1
reverselookup        

{0: 'grab',
 1: 'palm',
 2: 'super',
 3: 'shoot',
 4: 'Five',
 5: 'yoyo',
 6: 'aim',
 7: 'none',
 8: 'fist',
 9: 'peace'}

In [0]:
IMG_SAVE_PATH = r'drive/My Drive/Data/BGR'
NUM_CLASSES = len(CLASS_MAP)

In [0]:

def mapper(val):
    return CLASS_MAP[val]


def get_model():
    model = Sequential([
        SqueezeNet(input_shape=(227, 227, 3), include_top=False),
        Dropout(0.5),
        Convolution2D(NUM_CLASSES, (1, 1), padding='valid'),
        Activation('relu'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])
    return model

In [0]:
# load images from the directory
dataset = []
for directory in os.listdir(IMG_SAVE_PATH):
    path = os.path.join(IMG_SAVE_PATH, directory)
    if not os.path.isdir(path):
        continue
    for item in os.listdir(path):
        # to make sure no hidden files get in our way
        if item.startswith("."):
            continue
        img = cv2.imread(os.path.join(path, item))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (227, 227))
        dataset.append([img, directory])
    print(directory,"compleated !")    

'''
dataset = [
    [[...], 'grab'],
    [[...], 'palm'],
    ...
]
'''

grab compleated !
palm compleated !
super compleated !
shoot compleated !
Five compleated !
yoyo compleated !
aim compleated !
none compleated !
fist compleated !
peace compleated !


"\ndataset = [\n    [[...], 'rock'],\n    [[...], 'paper'],\n    ...\n]\n"

In [0]:


data, labels = zip(*dataset)
labels = list(map(mapper, labels))


'''
labels: grab,palm,aim,shoot,peace...
one hot encoded: [1,0,0], [0,1,0], [0,1,0], [0,0,1], [1,0,0]...
'''
# one hot encode the labels
labels = np_utils.to_categorical(labels)

In [0]:
# define the model
model = get_model()
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)






3039232/3032184 [==============================] - 0s 0us/step







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
# start training
model.fit(np.array(data), np.array(labels), epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
5000/5000 [==============================] - 18s 4ms/step - loss: 0.4027 - acc: 0.8800
Epoch 2/10
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0445 - acc: 0.9846
Epoch 3/10
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0250 - acc: 0.9920
Epoch 4/10
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0050 - acc: 0.9992
Epoch 5/10
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0067 - acc: 0.9982
Epoch 6/10
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0017 - acc: 0.9998
Epoch 7/10
5000/5000 [==============================] - 9s 2ms/step - loss: 6.2521e-04 - acc: 1.0000
Epoch 8/10
5000/5000 [==============================] - 9s 2ms/step - loss: 2.1995e-04 - acc: 1.0000
Epoch 9/10
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0027 - acc: 0.9990
Epoch 10/10
5000/5000 

In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
squeezenet (Model)           (None, 13, 13, 512)       722496    
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 512)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 10)        5130      
_________________________________________________________________
activation_1 (Activation)    (None, 13, 13, 10)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 10)                0         
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 727,626
Trainable params: 727,626
Non-trainable params: 0
________________________________________________

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()                 
drive = GoogleDrive(gauth)

In [0]:
model.save('handgesture_model_sqzNET.h5')
model_file = drive.CreateFile({'title' : 'handgesture_model_sqzNET.h5'}) 
model_file.SetContentFile('handgesture_model_sqzNET.h5')                       
model_file.Upload()

In [0]:
# download to google drive                      
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1yixGRv4FWiSIerg6wnwws84Wd9zrSye5'})